## OpenAI (ChatGPT)

Beispiel um gezielt Inhalte aus Vector Stores auszulesen.

Zuerst muss der Open AI API Key erstellt und ein Guthaben bereitgestellt werden:
* [API Key erstellen](https://platform.openai.com/account/api-keys)
* [Guthaben bereitstellen](https://platform.openai.com/settings/organization/billing/overview)

Mittels API Key erstellen wir den `client` welcher die Verbindung zum OpenAI API herstellt.


In [ ]:
import os
from openai import OpenAI
import json
from nbconvert import MarkdownExporter
from nbconvert.preprocessors import ExtractOutputPreprocessor
from IPython.display import Markdown, display
from datetime import datetime

key = ''
client = OpenAI(api_key=key)

**Anzeige der Vector Stores**

Als erstes Schauen wir welche Vector Stores vorhanden sind.

In [ ]:
try:
    all_vector_stores = []
    after = None

    while True:
        if after:
            response = client.vector_stores.list(limit=100, after=after)
        else:
            response = client.vector_stores.list(limit=100)

        data = response.data
        all_vector_stores.extend(data)

        # Wenn weniger als 100 Ergebnisse => keine nächste Seite
        if len(data) < 100:
            break

        # ID des letzten Eintrags als nächsten Cursor setzen
        after = data[-1].id

    if all_vector_stores:
        print("Deine vorhandenen Vector Stores:")
        for vs in all_vector_stores:
            print(f"- ID: {vs.id}, Name: {vs.name}, Created At: {vs.created_at}")
            print(f"  - Dokumente im Vector Store: {vs.file_counts}")

        print(f"\n📊 Gesamtanzahl Vector Stores: {len(all_vector_stores)}")
    else:
        print("Es wurden keine Vector Stores gefunden.")

except Exception as e:
    print(f"Fehler beim Auflisten der Vector Stores: {e}")
   

***

### Abfrage

Mittels zwei Hilfsfunktionen fragen wir unser Wissen ab:
- `get_vector_store_id_by_name` - Gibt die ID eines Vector Stores anhand seines Namens zurück.
- `query_vector_store` - Führt eine semantische Suche im Vector Store durch.

In [ ]:
def get_vector_store_id_by_name(target_name):
    try:
        after = None
        target_name_lower = target_name.lower()

        while True:
            # Abrufen mit oder ohne Cursor
            if after:
                response = client.vector_stores.list(limit=100, after=after)
            else:
                response = client.vector_stores.list(limit=100)

            for vs in response.data:
                if vs.name.lower() == target_name_lower:
                    return vs.id

            # Wenn weniger als 100 Ergebnisse => keine weitere Seite
            if len(response.data) < 100:
                break

            # Cursor für nächste Seite setzen
            after = response.data[-1].id

        print(f"Kein Vector Store mit dem Namen '{target_name}' gefunden.")
        return None

    except Exception as e:
        print(f"Fehler beim Abrufen der Vector Stores: {e}")
        return None

def query_vector_store(vector_store_id, query, top_k=3):
    try:
        response = client.responses.create(
            model="gpt-4o-mini",
            input=query,
            tools=[{
                "type": "file_search",
                "vector_store_ids": [vector_store_id]
            }]
        )
        return response
    except Exception as e:
        print(f"Fehler bei der Suche im Vector Store: {e}")
        return None
    

Dann folgt die eigentliche Abfrage:
- `vector_store_name`- Name des Vector Stores
- `user_input`- die eigentliche Frage

In [ ]:
vector_store_name = "m300-BiVo2021"
user_input = "was wird in diesem modul unterrichtet?"

# Führe eine Suche im Vector Store durch
vector_store_id = get_vector_store_id_by_name(vector_store_name)
response = query_vector_store(vector_store_id, user_input)

# Extrahiere den Markdown-Text aus der Antwort
if response:
    # response.output enthält eine Liste mit Nachrichten
    for item in response.output:
        if item.type == "message" and hasattr(item, "content"):
            for content_item in item.content:
                if content_item.type == "output_text" and hasattr(content_item, "text"):
                    markdown_text = content_item.text
                    display(Markdown(markdown_text))
                    



---

### Debug 

In [ ]:
# Formatiertes JSON anzeigen (z. B. zum Debuggen)
if response:
    # Versuche, das Response-Objekt in ein dict zu konvertieren, falls nötig
    if hasattr(response, 'model_dump'):
        response_dict = response.model_dump()  # bei pydantic-basierten Objekten
    else:
        response_dict = response  # falls es schon ein dict ist

    # Formatiert anzeigen
    print(json.dumps(response_dict, indent=2, ensure_ascii=False))
